# import

In [1]:
import sys
import os
import os.path as osp
import torch
import cv2
from tqdm import tqdm
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import numpy as np

In [2]:
sys.path.append("/workspace/pyvino/pyvino/model/human_pose_estimation/human_3d_pose_estimator/pose_extractor/build/")

In [3]:
from pyvino.model.reidentification.person_reidentifier import PersonReidentifier

In [4]:
reid_model = PersonReidentifier()

2020-04-20 03:12:05,850 - pyvino.model.base_model.base_model - INFO - person-reidentification-retail-0076 on Linux
2020-04-20 03:12:05,857 - pyvino.model.base_model.base_model - INFO - The path to cpu_extension is /opt/intel/openvino/inference_engine/lib/intel64/libcpu_extension_avx2.so


# constant

# path

In [5]:
data_name = 'player'
data_dir = osp.join('../output', data_name, 'tracking')
save_dir = osp.join('../output', data_name, 'tracking')

# function

In [6]:
def listdirs(dir_path, name_condition=None):
    dir_names = os.listdir(dir_path)
    if name_condition is not None:
        dir_names = [dir_name for dir_name in dir_names if dir_name.startswith(name_condition)]
    dir_names.sort()
    return dir_names

In [7]:
def save_pickle(data, path):
    with open(path, 'wb') as f:
        pickle.dump(data, f)


def load_pickle(path):
    with open(path, 'rb') as f:
        data = pickle.load(f)
    return data

In [8]:
def l2_norm(x):
    x_l2_norm = np.linalg.norm(x,ord=2)
    x_l2_normalized = x / x_l2_norm
    return x_l2_normalized

In [9]:
def mean_feature(features):    
    num_feature = len(features)    
    features = features.sum(axis=0)
    features = features / num_feature
    features = l2_norm(features)
    return features

# main

In [13]:
games = listdirs(data_dir, 'game')
for game in games:
    print(game)
    game_dir = osp.join(data_dir, game)
    clips = listdirs(game_dir, 'Clip')
    for clip in tqdm(clips):
        clip_dir = osp.join(game_dir, clip)
        tracking_ids = listdirs(clip_dir)
        for tracking_id in tracking_ids:
            tracking_dir = osp.join(clip_dir, tracking_id, 'person_image')
            file_names = listdirs(tracking_dir)
            # label_names = [label for label in file_names if osp.splitext(label)[1]=='.json']
            image_names = [image for image in file_names if osp.splitext(image)[1]=='.jpg']
            
            features = []
            for image_name in image_names:
                mean_feature
                
                file_name = os.path.splitext(image_name)[0]
                image_path = osp.join(tracking_dir, image_name)                
                frame = cv2.imread(image_path)
                if frame is None:
                    print('frame is None')
                    continue
                
                feature = reid_model.compute(frame)
                feature_save_name = "{}.pkl".format(file_name)
                feature_save_path = osp.join(save_dir, game, clip, tracking_id, 'feature', feature_save_name)
                os.makedirs(osp.dirname(feature_save_path), exist_ok=True)
                save_pickle(feature, feature_save_path)
                features.append(feature)
                
            features = np.array(features)
            mean_f = mean_feature(features)
            mean_feature_save_path = osp.join(save_dir, game, clip, tracking_id, 'mean_feature', 'mean_feature.pkl')
            os.makedirs(osp.dirname(mean_feature_save_path), exist_ok=True)
            save_pickle(mean_f, mean_feature_save_path)


  0%|          | 0/13 [00:00<?, ?it/s]

game1


  0%|          | 0/12 [00:00<?, ?it/s]

game10


  0%|          | 0/8 [00:00<?, ?it/s]

...

  0%|          | 0/9 [00:00<?, ?it/s]

game8


  0%|          | 0/9 [00:00<?, ?it/s]

game9


100%|██████████| 9/9 [01:46<00:00, 11.84s/it]
